# [Advent of Code - 2017](https://adventofcode.com/2017)

Advent of code is a puzzle solving website, two puzzles released for each day of advent (Dec 1st - Dec 25th). If previous years are anything to go by the cover a large variety of algorithms and are generally quite fun!

Each year the puzzels are built around a central theme, with this year's theme being that we have been "digitized" into a computer, and must solve various problems from inside the machine.

# Day 0

This portion contains various common pieces of code that'll be used on multiple days.

In [175]:
from collections import Counter, defaultdict, deque
from functools import reduce
from itertools import cycle, count, islice
from io import StringIO
import numpy as np
import os
import re


cat = ''.join


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

        
def tail(n, iterable):
    "Return an iterator over the last n items"
    # tail(3, 'ABCDEFG') --> E F G
    return iter(deque(iterable, maxlen=n))


def head(n, iterable):
    """Return an iterator over the first n items."""
    return iter(islice(iterable, n))


def nth_item(n, iterable):
    """Take the nth item from an iterator."""
    return next(islice(iterable, n, n + 1))


def last(iterable):
    """Returns the final item in an iterable."""
    return next(tail(1, iterable))


def first(iterable, n=1):
    return islice(iterable, n)


def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2017/input{}.txt'.format(day))
    return open(filename)


def neighbours4(x, y):
    return (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)


def neighbours8(x, y):
    return (
        (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1),
        (x - 1, y - 1), (x - 1, y + 1), (x + 1, y - 1), (x + 1, y + 1)
    )


def manhattan_distance(point1, point2):
    return sum(
        abs(a - b) for a, b in zip(point1, point2)
    )

# TREES

class Node(object):
    def __init__(self, name):
        self.name = name
        self._children = []
        self.parent = None
        
    def __repr__(self):
        return '<Node 0x:{} name={}>'.format(
            id(self),
            self.name
        )
    
    @property
    def children(self):
        """Immutable set of the node's children."""
        return tuple(self._children)
    
    def add_child(self, child):
        self._children.append(child)
    
    def remove_child(self, child):
        self._children.remove(child)
    
    def add_parent(self, parent):
        if self.parent:
            self.parent.remove_child(self)

        self.parent = parent

        if self.parent:
            self.parent.add_child(self)
    
    @property
    def root(self):
        if not self.parent:
            return self
        return self.parent.root
    
    @property
    def decendants(self):
        """Visit all children."""
        iterator = visit_pre_order(self)
        # Skip this node
        next(iterator)
        return iterator
    
    @property
    def siblings(self):
        for child in self.parent.children:
            if child == self:
                continue
            yield child

    
def visit_pre_order(node):
    """Visit a Tree in pre-order.
    
      A
     / \
    B   C
    
    A -> B -> C
    """
    yield node
    for child in node.children:
        yield from visit_pre_order(child)

# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

We're greeted by a door, and must proove that we are _not_ human to continue. The first puzzle has us performing a captha that "only a computer" can solve. We're required to sum digits in a list where each digit matches the one immediately following it, wrapping to the start if we overflow.

In [ ]:
def sum_if_match(nums, jump_distance):
    total = 0
    for index, n in enumerate(nums):
        next_n = nums[(index + jump_distance) % len(nums)] 
        if n == next_n:
            total += n
    return total

def sum_consecutive(data):
    nums = list(map(int, data))
    jump_distance = 1
    return sum_if_match(nums, jump_distance)


assert sum_consecutive('1122') == 3
assert sum_consecutive('1111') == 4
assert sum_consecutive('1234') == 0
assert sum_consecutive('91212129') == 9

sum_consecutive(Input(1).read().strip())

For the second portion, we're again summing digits, but now only if we match the digit exactly _half the list_ away. At this point we can modify the initial code and provide a jump distance.

In [ ]:
def sum_half(data):
    nums = list(map(int, data))
    jump_distance = len(nums) // 2
    return sum_if_match(nums, jump_distance)
    

assert sum_half('1212') == 6
assert sum_half('1221') == 0
assert sum_half('123425') == 4
assert sum_half('123123') == 12
assert sum_half('12131415') == 4

sum_half(Input(1).read().strip())

# [Day 2: Corruption Checksum](https://adventofcode.com/2017/day/2)

Here we're required to perform some data anaylsis on a spreadsheet calculating a checksum of each row by finding the difference of the max and min values.

In [ ]:
def parse_input(data):
    as_ints = []
    for row in data.split('\n'):
        if not row:
            break
        nums = list(map(int, re.findall('\d+', row)))
        as_ints.append(nums)
    return as_ints


data = parse_input(Input(2).read())

sum((max(row) - min(row) for row in data))

The second portion requires us to find pairs of number in each row that are evenly divisible, and summing the result of their division. As we don't know in which order the pair will appear, I sort each row when meeting it.

In [ ]:
def sum_even_div(data):
    total = 0
    for nums in data:
        nums = sorted(nums)
        for i, x in enumerate(nums[:-1]):
            for y in nums[i + 1:]:
                if y % x == 0:
                    total += y // x
                    break
    return total


sum_even_div(data)

# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

We need to find the coordinate of an number when it's displayed in a spiral format. I.e.

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

I started this problem by working out an equation to find the location of the Nth element, without building the rest of the grid. While this worked well for the first part of the problem, the second part requires us to build a spiral grid anyway! (It's much smaller, but still.)

In [ ]:
def find_coorindates(N):
    """Find the coordinates of N in a spiral matrix.
    
    We can find the shell that the number occurs in by
    finding the upper limit for each shell and 
    """
    # First find the shell that the number appears in
    # number of elements in each shell is 4*(n-1)
    if N == 1:
        return 0, 0

    shell = 1
    shell_size = 1
    while N > shell_size ** 2:
        shell += 1
        shell_size += 2

    shell_end = shell_size ** 2
    shell_start = (shell_size - 2) ** 2
    elms_in_shell = shell_end - shell_start
    position_in_shell = N - shell_start
    
    side_length = elms_in_shell // 4
    half_side = side_length // 2
    
    side = position_in_shell / elms_in_shell
    
    if side <= 0.25:
        # right
        x = half_side
        y = (position_in_shell % side_length) - half_side
    elif side <= 0.5:
        # top
        x = (position_in_shell % side_length) - half_side
        y = half_side
    elif side <= 0.75:
        # left
        x = -half_side
        y = (position_in_shell % side_length) - half_side
    else:
        # bottom
        x = (position_in_shell % side_length) - half_side
        y = -half_side
    return (x, y)
    
    
def carry_distance(N):
    x, y = find_coorindates(N)
    return manhattan_distance((0, 0), (x, y))


data = 312051
assert carry_distance(1) == 0
assert carry_distance(12) == 3
assert carry_distance(23) == 2
assert carry_distance(1024) == 31
carry_distance(data)

The second part requires us to perform a "stress test" to populate a spiral grid with values the sum of all adjacent cells in the grid. As we don't know the upper bound (and I don't know how to initialize an infinite grid that can be referenced arbitarily..) I went with a dictionay to store point references and their values. 

By combining two infinite generators that cycle through the directions we turn and the distances we need to travel, we build an a third infinite generator that contains all the steps we'll take.

In [ ]:
from itertools import cycle, count

def spiral_distances():
    """"Yields 1, 1, 2, 2, 3, 3, ...
    
    As the spiral wraps around itself, we increase
    the distance we travel by 1 every two distances.
    
    This is because every 2 distances we're moving in
    the opposite direction, so to we increase the
    distance to ensure we can move past the movement
    we're now opposing.
    """
    for distance in count(1):
        yield distance
        yield distance

            
def directions():
    """Yields R, U, L, D, R, U, L, D, ..."""
    up = (0, -1)
    down = (0, 1)
    left = (-1, 0)
    right = (1, 0)
    return cycle((right, up, left, down))


def spiral_movements():
    for distance, direction in zip(spiral_distances(), directions()):
        for _ in range(distance):
            yield direction


def stress_test(max_val):
    grid = {}
    x, y = 0, 0
    grid[(x, y)] = 1
    for direction in spiral_movements():
        dx, dy = direction
        x += dx
        y += dy
        val = sum(
            grid.get(neighbour, 0)
            for neighbour in neighbours8(x, y)
        )

        grid[(x, y)] = val

        if val > max_val:
            return val
stress_test(data)
    

# [Day 4: High-Entropy Passphrases](https://adventofcode.com/2017/day/4)

W're tasked with checking the validity of a series of passphrases. A passphrase is considered valid if each word within the passphrase is unique.

In [ ]:
def is_valid(passphrase):
    words = re.findall('\w+', passphrase)
    unique = set(words)
    return len(unique) == len(words)

def valid_passphrases(data):
    return sum(
        is_valid(passphrase)
        for passphrase in data.split('\n')
    )
    

valid_passphrases(Input(4).read())

Part two requires us to ensure that no anagrams are present.

In [ ]:
def is_valid(passphrase):
    words = re.findall('\w+', passphrase)
    unique = set(
        cat(sorted(w)) for w in words
    )
    return len(unique) == len(words)

assert not is_valid('abcde xyz ecdab')

valid_passphrases(Input(4).read())

# [Day 5: A Maze of Twisty Trampolines, All Alike](https://adventofcode.com/2017/day/5)

In this example we're following a series of jump instructions through an list that modify the jump distance once it's been performed. For each item in the list, `n`, we jump `n` steps away and increment the jump value in at that index by 1.

Originally I didn't have the `increment_by` as a callable, and in doing so the program is noticably slowed (part two went from 7s to 9s), however it makes the second part trivial so I introduced it.

In [ ]:
def follow_instructions(data, increment_by=lambda n: 1):
    instructions = list(map(int, re.findall('-?\d+', data)))
    step_count = 0
    index = 0
    while True:
        try:
            jump = instructions[index]
        except IndexError:
            break
        instructions[index] += increment_by(jump)
        index += jump
        step_count += 1
    return step_count

test_input = """
0
3
0
1
-3
"""
assert follow_instructions(test_input) == 5

% time follow_instructions(Input(5).read())

The second part is the same as the first but this time we decrement the jump value if it's `>= 3`.

In [ ]:
increment_by = lambda n: -1 if n > 2 else 1
assert follow_instructions(test_input, increment_by) == 10
% time follow_instructions(Input(5).read(), increment_by)

# [Day 6: Memory Reallocation](https://adventofcode.com/2017/day/6)

We're tasked re-bucketing data, and dedecting any cyclical allocations. Given N buckets, take the maximum, set that bucket to zero and then add one for N to each other bucket in order, wrapping to the start.

i.e.

```
    (0, 2, 7, 0) Initial state, 7 is highest so set to zero and distribute
    (2, 4, 1, 2) 4 is now the highest
    (3, 1, 2, 3) We have two threes, the first takes precedence
    (0, 2, 3, 4) 4 is now the hightest
    (1, 3, 4, 1) 4 again wins
    (2, 4, 1, 2) *LOOP* We've seen this state before, and was detected on the 5th step.
```

As I'm generating this data as we go, I don't think there's any nifty algorithms that we can implement to detect a loop. Instead I just keep track of what's been seen and compare the current state.

As we want to use a set to keep track of what we've seen there's a lot of conversion between lists and tuples, but this is a smaller trade off as the seen list can get quite large, so [O(1) member test compared to O(N)](https://wiki.python.org/moin/TimeComplexity) wins! (Using a list here takes ~20 times longer.)

In [ ]:
def redistribute_memory(banks):
    seen = set()
    
    for cycle in count(1):
        if banks in seen:
            break
        seen.add(banks)
        banks = redistribute(banks)
    return cycle - 1


def redistribute(banks):
    banks = list(banks)
    blocks = max(banks)
    max_index = banks.index(blocks)
    
    banks[max_index] = 0
    
    offset = max_index + 1
    addition = blocks // len(banks)
    extra_lim = blocks % len(banks)
    
    for i in range(len(banks)):
        idx = (offset + i) % len(banks)
        banks[idx] += addition
        if i < extra_lim:
            banks[idx] += 1
    return tuple(banks)

In [ ]:
assert redistribute((0, 2, 7, 0)) == (2, 4, 1, 2)
assert redistribute_memory((0, 2, 7, 0)) == 5

In [ ]:
banks = tuple(
    int(num)
    for num in re.findall('\d+', Input(6).read())
)
redistribute_memory(banks)

For part two we need to know the length of the loop. We can do this by keeping track of the original index of each memory bank and compare it to the index we're on when we reach the final bank.

In [ ]:
def redistribute_memory2(banks):
    seen = {}
    
    for cycle in count(1):
        if banks in seen:
            break
        seen[banks] = cycle
        banks = redistribute(banks)
    return cycle - seen[banks] - 1

In [ ]:
redistribute_memory2(banks)

# [Day 7: Recursive Circus](http://adventofcode.com/2017/day/7)

Here we meet our first tree. This could probably be solved without such large data structures, but this may come up later so I think it's worth investing time into now.

In [ ]:
class WeightedNode(Node):
    def __init__(self, name, weight):
        self.weight = weight
        super().__init__(name)
        
    @property
    def decendant_weight(self):
        return sum(child.weight for child in self.decendants)
    
    @property
    def total_weight(self):
        return self.weight + self.decendant_weight


def parse_input(data):
    chunks = re.findall("(\w+) \((\d+)\)(?: -> ([a-z, ]+))?", data)
    
    nodes = {
        name: WeightedNode(name, int(weight))
        for name, weight, _
        in chunks
    }
    
    for name, _, children in chunks:
        if not children:
            continue
        node = nodes[name]
        for child_name in children.split(', '):
            nodes[child_name].add_parent(node)

    # We can use any node here.
    return node.root

In [ ]:
test_input = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""

test_root = parse_input(test_input)
assert test_root.name == 'tknk'

In [ ]:
root = parse_input(Input(7).read())
root.name

For the second part I needed to find the node lowest in the tree that has a different total weight than it's siblings and calculate what the weight _should_ be. As we know there's only a single node that's the problem, from the root we can identify the node that's incorrect and focus on that subtree until it's no longer incorrect.

In [ ]:
def find_total_weights(node):
    while True:
        weights = [
            child.total_weight
            for child in node.children
        ]
        if len(set(weights)) == 1:
            # No problems, all weights are the same.
            # The problematic node was in the previous
            # iteration.
            break
        
        # There are two distinct values, take the second
        # most common to give us the odd one out.
        incorrect_value = Counter(weights).most_common(2)[1][0]
        incorrect_index = weights.index(incorrect_value)
        node = node.children[incorrect_index]

    target_weight = next(node.siblings).total_weight
    diff = target_weight - node.total_weight
    return node.weight + diff


find_total_weights(root)

# [Day 8: I Heard You Like Registers](http://adventofcode.com/2017/day/8)

For this puzzle we have to perform a series of conditional operations based on the values of some "register". All of python's conditional operations call magic methods under the hood. i.e. `A > B` is transformed to `A.__gt__(b)`. These operators are mapped to functions within the operator module.

As we don't know the number of registers that will be referenced, I use a default dict that can grow to whatever's required. The addition of the `__MAX__` flag is for the second part of the question, which wants to know what the largest value seen was.

In [ ]:
from operator import gt, lt, ge, le, eq, ne

def perform_instructions(instructions):
    instructions = list(parse_instructions(instructions))
    regs =  defaultdict(int)
    
    for reg, op, amount, test in instructions:
        test_reg, test_op, test_val = test
        
        if not test_op(regs[test_reg], test_val):
            # Test failed, skip operation
            continue
            
        if   op == 'inc': regs[reg] += amount
        elif op == 'dec': regs[reg] -= amount
            
        if regs[reg] > regs['__MAX__']:
            regs['__MAX__'] = regs[reg]

    return regs


def parse_instructions(instructions):
    op_map = {
        '>': gt,
        '<': lt,
        '>=': ge,
        '<=': le,
        '==': eq,
        '!=': ne
    }
    for d in re.findall('(\w+) (\w+) (-?\d+) if (\w+) ([><=!]+) (-?\d+)', instructions):
        reg, op, amount, test_reg, test_op, test = d
        yield (
            reg, op, int(amount), (test_reg, op_map[test_op], int(test))
        )
       
    
max_reg_val = lambda regs: max(
    val
    for key, val in regs.items()
    if key != '__MAX__'
)

In [ ]:
test_instructions = """
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
"""
regs = perform_instructions(test_instructions)
assert max_reg_val(regs) == 1

In [ ]:
regs = perform_instructions(Input(8).read())
max_reg_val(regs)

In [ ]:
regs['__MAX__']

# [Day 9: Stream Processing](http://adventofcode.com/2017/day/9)

This problem asks us seperate blocks according to the following rules:

* A new group is identified by a `{` character
* The most recently opened group is closed by a `}` character
* Garbage is identified by a `<` character
* Within garbage, all charactesr have no meaning onther than `!` and `>`
* Within garbage `!` causes the next character to be ignored
* Within garbage `>` identifies the end of the gabage
* Each group can contain subgroups

I chose to use a stream interface for this problem as we can recursively solve and update our position within the steam without having to parse it. Perhaps a nicer solution would be to parse the input into the tree it represents.

In [ ]:
def garbage_advance(stream):
    garbage = 0
    while True:
        char = stream.read(1)
        if char == '!':
            # Ignore the next character
            stream.read(1)
            continue

        if char == '>':
            return garbage
        garbage += 1


def sum_groups(stream, depth=0):
    block_total = depth
    garbage_total = 0

    while True:
        char = stream.read(1)
        
        if not char:
            # Reached the end of the input
            break
            
        if char == '<':
            # We have garbage, advance to the end
            garbage_total += garbage_advance(stream)
            continue

        if char == '{':
            # Start of the next block
            sub_block_total, sub_garbage_total = sum_groups(stream, depth + 1)
            block_total += sub_block_total 
            garbage_total += sub_garbage_total

        if char == '}':
            # End of current block
            return block_total, garbage_total
    return block_total, garbage_total

    
assert sum_groups(StringIO('{}'))[0] == 1
assert sum_groups(StringIO('{{}, {}}'))[0] == 5
assert sum_groups(StringIO('{{{}}}'))[0] == 6
assert sum_groups(StringIO('{{<a!>},{<a!>},{<a!>},{<ab>}}'))[0] == 3
assert sum_groups(StringIO('{{<!!>},{<!!>},{<!!>},{<!!>}}'))[0] == 9

assert garbage_advance(StringIO('>')) == 0
assert garbage_advance(StringIO('random characters>')) == 17
assert garbage_advance(StringIO('{o"i!a,<{i<a>')) == 10

sum_groups(Input(9))

# [Day 10: Knot Hash](http://adventofcode.com/2017/day/10)

For this puzzle we are required to implement a custom hashing function, which is based entirely on flipping the order of a range of nodes in a circularly linked list. To avoid complications when flipping a range that wraps a list, I rotate the list by the offset of the next rotation, reverse the range (as it is now guaranteed not to wrap) and rotate again by the remaining amount, resetting our offset.

(This method was updated after unlocking part two for the inclusion of rounds.)

In [ ]:
def knot_hash_round(seq, lengths):
    skip_size = 0
    pos = 0

    for length in lengths:
        # partially rotate the list
        seq = seq[pos:] + seq[:pos]

        # perform the reversal
        seq[:length] = seq[:length][::-1]

        # complete the rotation
        seq = seq[len(seq) - pos:] + seq[:len(seq) - pos]

        # Update state
        pos += length + skip_size
        pos = pos % len(seq)
        skip_size += 1
    return seq

In [ ]:
test_input = list(range(5))
res = knot_hash_round(test_input, [3, 4, 1, 5])
assert(res[0] * res[1]) == 12

In [ ]:
data = "199,0,255,136,174,254,227,16,51,85,1,2,22,17,7,192"
lengths = map(int, data.split(','))
seq = list(range(256))
res = knot_hash_round(seq, lengths)
print(res[0] * res[1])

Part two has quite drastic changes. 

1. Perform 64 rounds, maintaing state
* Input is no longer a CSV, use character ordinal value instaed
* Add a trailer of [17, 31, 73, 47, 23] to the lengths
* Reduce results to 16 integers by XORing in blocks of 16
* Translate resulting integers into a hexidecimal digest

To perform the 64 rounds I create a generator that repeats the lengths 64 times. This means we don't need to modify our previous code and the state is automatically preserved.

In [ ]:
# from itertools import chain, repeat
from operator import xor

def knot_hash(string):
    lengths = list(map(ord, string))
    seq = list(range(256))
    trailer = [17, 31, 73, 47, 23]
    lengths.extend(trailer)
    # Expand the lengths to represent each round
    lengths = chain(*repeat(lengths, 64))

    sparse = knot_hash_round(seq, lengths)
    dense = [
        reduce(xor, chunk)
        for chunk in chunks(sparse, 16)
    ]
    return cat(
        '{0:0{1}x}'.format(c, 2) for c in dense
    )

In [ ]:
knot_hash(data)

# [Day 11: Hex Ed](http://adventofcode.com/2017/day/11)

I've never worked with hexagonal grids before, so this was quite a learning experience for me. Thankfully I found [this](https://www.redblobgames.com/grids/hexagons/), which contains a beautiful breakdown of the various techniques that can be used when working with them.

A hexagonal grid can be mapped to cartesian coordinates in a number of ways. Perhaps the hardest to comprehend yet the most useful for us is using cube coordinates. The article above lists in detail how a hexagon can be represented as a slice of a three-dimentional cube. Each face re

In [ ]:
deltas = {
    'nw': (-1, 1, 0),
    'n': (0, 1, -1),
    'ne': (1, 0, -1),
    'sw': (-1, 0, 1),
    's': (0, -1, 1),
    'se': (1, -1, 0)
}


def hex_manhattan_distance(point1, point2=(0, 0, 0)):
    return manhattan_distance(point1, point2) // 2


def follow_path(path):
    origin = (0, 0, 0)
    x, y, z = origin
    max_dist = 0
    for step in path.split(','):
        dx, dy, dz = deltas[step]
        x += dx
        y += dy
        z += dz

        yield (x, y, z)

In [ ]:
data = Input(11).read().strip()
points = follow_path(Input(11).read().strip())
hex_manhattan_distance(last(points))

In [ ]:
points = follow_path(data)
max(map(hex_manhattan_distance, points))

# [Day 12: Digital Plumber](https://adventofcode.com/2017/day/12)

Today we're required to build a bidirectional graphs. We're supplied list of nodes of the form:

```
    12 <-> 34, 56, 78
```

Which reads, `node 12 is connected to node 34, 56 and 78`. These nodes are in no particular order, so we can't know before building the graphs how many we'll end up with (the last node could connect all of them together). 

In [ ]:
def build_graph(data):
    graph = defaultdict(set)
    for node, connected in re.findall('(\d+) <-> ([\d, ]+)', data):
        connected = connected.split(', ')
        for dest in connected:
            graph[node].add(dest)
            graph[dest].add(node)
    return graph


def count_connected_to_zero(graph):
    groups = connected_nodes(graph)
    for group in groups:
        if '0' in group:
            return len(group)


def connected_nodes(graph):
    groups = []
    processed = set()

    for node in graph.keys():
        if node in processed:
            # Seen before, skip
            continue

        # Follow all new connections
        group = set()
        to_investigate = [node, ]
        while to_investigate:
            node = to_investigate.pop() 
            if node in group:
                continue

            to_investigate.extend(graph[node])

            group.add(node)
            processed.add(node)
            
        groups.append(group)
    return groups

In [ ]:
test_input = """
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
"""

test_graph = build_graph(test_input)
assert(count_connected_to_zero(test_graph)) == 6

In [ ]:
graph = build_graph(Input(12).read())
count_connected_to_zero(graph)

In [ ]:
assert(len(connected_nodes(test_graph))) == 2

In [ ]:
len(connected_nodes(graph))

# [Day 13: Packet Scanners](https://adventofcode.com/2017/day/13)

Today we're required to infiltrate a firewall patrolled by security scanners. Each security scanner works at a distinct layer within the wall and walks back and forth along a line of a given depth. We pass through at depth 0, if the the bot is at this depth as we are passing through we're caught.

So, if each bot is walking a line of depth `d`, then there are `2d - 2` locations it moves between (the ends are only hit once per cycle). So at a given step in the cycle the bot will be at position `(2d - 2) % step`.

In [ ]:
def build_firewall(data):
    """Parse input and translate depths to number of locations."""
    firewall = defaultdict(int)
    
    for layer, depth in re.findall('(\d+): (\d+)', data):
        depth = int(depth)
        firewall[int(layer)] = 2 * depth - 2
    return firewall


def walk_firewall(firewall, delay=0):
    length = max(firewall.keys())
    
    for step in range(length + 1):
        depth = firewall[step]
        
        if depth > 0:
            iteration = step + delay
            bot_position = iteration % depth
            
            if bot_position == 0:
                cost = step * (depth + 2) // 2
                yield cost

In [ ]:
test_input = """
0: 3
1: 2
4: 4
6: 4
"""
test_firewall = build_firewall(test_input)
assert sum(walk_firewall(test_firewall)) == 24

In [ ]:
firewall = build_firewall(Input(13).read())
sum(walk_firewall2(firewall))

In [ ]:
def first_non_hit(firewall):
    for n in count(0):
        for cost in walk_firewall(firewall, n):
            # We were caught, so we can discard this value
            break
        else:
            return n

In [ ]:
assert first_non_hit(test_firewall) == 10

In [ ]:
% time first_non_hit(firewall)

# Day 14

In [ ]:
USED = '1'


def to_binary(val):
    res = cat(
        bin(int(c, 16))[2:].zfill(4)
        for c in val
    ).ljust(32, '0')
    return res
    
    
def to_grid(key):
    rows = []
    for n in range(128):
        hash = knot_hash('{}-{}'.format(key, n))
        rows.append(to_binary(hash))
    return rows
        

assert to_binary('a0c2017') == '10100000110000100000000101110000'
test_grid = to_grid('flqrgnkx')
grid = to_grid('amgozmfv')

# Find the total number of used values
sum(map(lambda row: Counter(row)[USED], grid))

In [ ]:
def find_regions(grid):
    graph = defaultdict(list)
    for x in range(128):
        for y in range(128):
            if grid[x][y] != USED:
                # Empty location, don't care about it.
                continue

            # Add initial region, each node is connected
            # to itself
            graph[(x, y)].append((x, y))
            
            # As we're iterating over a grid, we only need
            # to check N and E neighbours as others will
            # have already been seen
            to_check = [(x + 1, y), (x, y + 1)]
            for x2, y2 in to_check:
                off_grid = x2 == 128 or y2 == 128
                if off_grid:
                    # Invalid location, ignore
                    continue

                if grid[x2][y2] == USED:
                    graph[(x, y)].append((x2, y2))
                    graph[(x2, y2)].append((x, y))

    connected = connected_nodes(graph)
    return connected


assert len(find_regions(test_grid)) == 1242
len(find_regions(grid))

# Day 15

In [ ]:
def build_generator(seed, factor, rule=1):
    prev = seed
    val = seed
    
    while True:
        prev = (prev * factor) % 2147483647

        if prev % rule == 0:
            yield prev


def gen_pairs(seed_a, seed_b):
    yield from zip(
        build_generator(seed_a, 16807),
        build_generator(seed_b, 48271)
    )


def judge(pairs, limit):
    pairs = islice(pairs, 0, limit)
    trailing_bits = lambda n: n & (2 ** 16 - 1)
    return sum(
        trailing_bits(a) == trailing_bits(b)
        for n, (a, b) in enumerate(pairs)
    )

In [ ]:
test_gen = gen_pairs(65, 8921)
# print(judge(test_gen, 5))
# assert judge(test_gen, 5) == 1
assert judge(test_gen, int(4e7)) == 588

In [ ]:
# Generator A starts with 516
# Generator B starts with 190
gen = gen_pairs(516, 190)
judge(gen, int(4e7))

In [ ]:
gen = gen_pairs(516, 190)
judge(gen, int(4e7))
%lprun -f build_generator judge(gen, 10000)

In [ ]:
def gen_pairs(seed_a, seed_b):
    while True:
        yield from zip(
            build_generator(seed_a, 16807, 4),
            build_generator(seed_b, 48271, 8)
        )

gen = gen_pairs(516, 190)
build_generator judge(gen, int(5e6))

# Day 16

In [ ]:
def parse_input(data):
    moves = []
    for chunk in data.split(','):
        op = chunk[0]
        if op == 's':
            args = [int(chunk[1:])]
        elif op == 'x':
            args = list(map(int, chunk[1:].split('/')))
        elif op == 'p':
            args = chunk[1:].split('/')
        moves.append((op, args))
    return moves
            
    

def follow_dance_moves(programs, moves):    
    for op, args in moves:
        if op == 's':
            rotation = args[0]
            programs = programs[-rotation:] + programs[:-rotation]
        elif op == 'x':
            x, y = args
            programs[x], programs[y] = programs[y], programs[x]
        elif op == 'p':
            a, b = args
            x, y = programs.index(a), programs.index(b)
            programs[x], programs[y] = programs[y], programs[x]
    return programs


test_moves = parse_input('s1,x3/4,pe/b')
assert cat(follow_dance_moves(list('abcde'), test_moves)) == 'baedc'
moves = parse_input(Input(16).read())
cat(follow_dance_moves(list('abcdefghijklmnop'), moves))

In [ ]:
programs = list('abcdefghijklmnop')
lim = 1000000000

positions = []
positions.append(cat(programs))
for n in range(lim):
    programs = follow_dance_moves(programs, moves)
    p = cat(programs)
    if p in positions:
        break
    positions.append(p)
    
# We have a repeating index, as such just we can find the end position easily
rem = lim % len(positions)
positions[rem]

# Day 17

In [ ]:
def spinlock(step_size):
    state = [0]
    pos = 0
    for step in count(1):
        pos = (pos + step_size) % len(state) + 1
        state.insert(pos, step)
        yield state, pos

def spin2017(step_size):
    items = head(
        2017,
        spinlock(step_size),
    )
    state, pos = list(items)[-1]
    print(pos)
    return state[pos + 1]
    

# gen = spinlock(3)
# assert next(gen)[0] == [0, 1]
# assert next(gen)[0] == [0, 2, 1]
# assert spin2017(3) == 638

spin2017(316)

In [ ]:
def track_val_at_index(step_size, index=1):
    state_length = 1
    pos = 0
    val = None
    for step in count(1):
        pos = (pos + step_size) % step + 1
        if pos == index:
            val = step
        state_length += 1
        yield val

%time nth_item(50000000, track_val_at_index(316))

# Day 18

In [ ]:
def parse_instructions(data):
    return re.findall('(\w+) (\w+)(?: ([\-\w]+))?', data)


def follow_instructions(instructions, regs, queue=[]):
    
    int_or_reg = lambda x: regs[x] if x.isalpha() else int(x)
    pos = 0
    played = None
    
    while pos < len(instructions):
        instruction, args = instructions[pos][0], instructions[pos][1:]
        
        if   instruction == 'set': regs[args[0]] = int_or_reg(args[1])
        elif instruction == 'add': regs[args[0]] += int_or_reg(args[1])
        elif instruction == 'snd': yield 'snd', int_or_reg(args[0])
        elif instruction == 'mul': regs[args[0]] *= int_or_reg(args[1])
        elif instruction == 'mod': regs[args[0]] %= int_or_reg(args[1])
        elif instruction == 'rcv': 
            while len(queue) == 0:
                yield ('rcv', )
            regs[args[0]] = queue.popleft()
        elif instruction == 'jgz':
            should_jump = int_or_reg(args[0]) > 0
            if should_jump:
                pos += int_or_reg(args[1])
                continue
                
        pos += 1

In [ ]:
def last_snt_at_first_rcv(instructions):
    gen = follow_instructions(instructions, defaultdict(int))

    sent = 0
    for op in gen:
        if op[0] == 'snd':
            sent = op[1]
        else:
            break
    return sent

test_instructions = """
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
"""
last_snt_at_first_rcv(parse_instructions(test_instructions))

In [ ]:
instructions = parse_instructions(Input(18).read())
last_snt_at_first_rcv(instructions)

In [ ]:
def perform_duet(instructions):
    regs_a = defaultdict(int)
    regs_a['p'] = 0
    queue_a = deque()

    regs_b = defaultdict(int)
    regs_b['p'] = 1
    queue_b = deque()
    
    gen_a = follow_instructions(instructions, regs_a, queue_a)
    gen_b = follow_instructions(instructions, regs_b, queue_b)
    
    c = 0
    
    while True:
        val_a = next(gen_a)
        val_b = next(gen_b)
        
        if val_a[0] == val_b[0] and val_a[0] == 'rcv':
            # deadlock
            break

        if val_a[0] == 'snd':
            queue_b.append(val_a[1])
            
        if val_b[0] == 'snd':
            queue_a.append(val_b[1])
            c += 1
            
    return c

test_instructions = parse_instructions("""
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
""")
assert perform_duet(test_instructions) == 3

In [ ]:
perform_duet(instructions)

# Day 19

This solution works but gets caught in a loop and needs work.

In [ ]:
DIRECTIONS = [
    (0, 1),
    (0, -1),
    (1, 0),
    (-1, 0),
]
import string

def follow_path(grid):
    y = 0
    x = grid[0].index('|')
    direction = (0, 1)
    chars = []
    
    d = 0
    steps = 0
    while True:
        dx, dy = direction
        x += dx
        y += dy
        
        if (x < 0 or y < 0):
            # Walked off the grid, we're done
            break
            
        steps += 1

        if char == '+':
            # Only change if there's no other option
            try:
                next_char = grid[y + dy][x + dx]
                should_change_dir = next_char == ' '
            except IndexError:
                should_change_dir = True
            
            if should_change_dir:
                # Changing direction
                for dx2, dy2 in DIRECTIONS:
                    if abs(dx2) == abs(dx):
                        # we have to change direction
                        continue
                    try:
                        next_char = grid[y + dy2][x + dx2]
                        if next_char == ' ':
                            # We have to stay on the path
                            continue
                    except IndexError:
                        continue

                    if abs(dy2) and next_char not in '|+' + string.ascii_uppercase:
                        continue
                    elif abs(dx2) and next_char not in '-+' + string.ascii_uppercase:
                        continue
                    direction = (dx2, dy2)
                    d += 1
                    break
                else:
                    print('no change')
        
        elif char not in '-| ':
            chars.append(char)
            print (cat(chars), steps)
    print('direction changes', d)
    return cat(chars)



test_grid = """     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
""".split('\n')

assert follow_path(test_grid) == 'ABCDEF'
print(Counter(Input(19).read()))
follow_path(Input(19).read().split('\n'))

# Day 20

We're given a set of particles, their position, velocity and accelleration. We need to find the particle that stays closest to the origin in the long term. This means that over a period of time, we need to know which ones are accelerating away from the origin the fastest, as eventually all the others will have moved futher away.

In [148]:
def parse_input(data):
    vector = r'(-?\d+,-?\d+,-?\d+)'
    results = re.findall(r'p=<{vector}>, v=<{vector}>, a=<{vector}>'.format(vector=vector), data)
    to_vector = lambda x: np.array([int(y) for y in x.split(',')])
    data = tuple(
        tuple(to_vector(c) for c in chunks)
        for idx, chunks in enumerate(results)
    )
    return data


def perform_update(state):
    return tuple(
        (p + v + a, v + a, a)
        for (p, v, a) in state
    )


def tick(state, remove_collisions=False):
    while True:
        yield state
        state = perform_update(state)
        
        if remove_collisions:
            counts = Counter([tuple(pos) for (pos, _, _) in state])
            state = tuple(
                (pos, vec, acc)
                for pos, vec, acc in state
                if counts[tuple(pos)] == 1
            )

            

def closest_to_zero(state):
    closest_particle = 0
    dist = lambda p: manhattan_distance((0, 0, 0), p)
    
    accellerations = [
        sum(map(abs, acc))
        for (_, _, acc) in state
    ]
    
    slowest = min(accellerations)
    slowest_particles = (
        particle
        for idx, particle in enumerate(state)
        if accellerations[idx] == slowest
    )
    closest_slow_particle = min(
        slowest_particles,
        key=lambda particle: dist(particle[0])
    )
    # Now we have the slowest particle, find its index.
    for idx, particle in enumerate(state):
        if np.allclose(particle, closest_slow_particle):
            return idx

        
test_data = """
p=<3,0,0>, v=<2,0,0>, a=<-1,0,0>
p=<4,0,0>, v=<0,0,0>, a=<-2,0,0>
"""
state = parse_input(test_data)
state = perform_update(state)
assert list(state[0][0]) == [4, 0, 0]
assert closest_to_zero(state) == 0

In [149]:
state = parse_input(Input(20).read())
closest_to_zero(state)

243

In [318]:
def no_collisions(state):
    for i, state in enumerate(tick(state, True)):
        if i > 100:
            break
    return len(state)          

no_collisions(state)

648

# Day 21

In [315]:
def to_array(data):
    return np.array(
        [
            [0 if char == '.' else 1 for char in row]
            for row in data.split('/')
        ],
        dtype=int
    )


def to_str(arr):
    return '\n'.join(
        cat('.#'[int(val)] for val in row)
        for row in arr
    )


def rotations(arr):
    for dist in range(4):
        yield np.rot90(arr, dist)
        
        
def flips(arr):
    yield arr
    yield np.flip(arr, 0)
    yield np.flip(arr, 1)

    
def parse_input(data):
    mapping = {}
    for lhs, rhs in re.findall(r'(.*) => (.*)', data):
        rhs_array = to_array(rhs)
        lhs_array = to_array(lhs)
        
        for flip in flips(lhs_array):
            for rot in rotations(flip):
                frozen_array = tuple(map(tuple, rot))
                mapping[frozen_array] = rhs_array
    return mapping


def chunkify_array(arr, size):
    return [
        arr[i:i+size, j:j+size] for j in range(0, len(arr), size)
        for i in range(0, len(arr), size)
    ]


def enhance(arr, mapping):
    if len(arr) % 2 == 0:
        size = len(arr) // 2
        new_size = size * 3
        chunks = chunkify_array(arr, 2)
        
    elif len(arr) % 3 == 0:
        size = len(arr) // 3
        new_size = size * 4
        chunks = chunkify_array(arr, 3)
        
    # Now we perform the mapping on each section, and rejoin
    # to a single array.
    out = np.zeros((new_size, new_size), dtype=int)
    for idx, chunk in enumerate(chunks):
        frozen_chunk = tuple(map(tuple, chunk))
        mapped = mapping[frozen_chunk]
        i = (idx * len(mapped)) % new_size
        j = (idx * len(mapped)) // new_size * len(mapped)
        out[i:i+len(mapped), j:j+len(mapped)] = mapped
        
    return out


def run(mapping, iterations):
    start = np.array(
    object=(
            (0, 1, 0),
            (0, 0, 1),
            (1, 1, 1),
        )
    )
    arr = start
    for _ in range(iterations):
        arr = enhance(arr, mapping)
        yield arr

        
def count_turned_on(mapping, iterations):
    result = last(run(mapping, iterations))
    return sum(sum(result))             


test_input = """
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
"""

mapping = parse_input(test_input)
assert count_turned_on(mapping, 2) == 12

In [311]:
mapping = parse_input(Input(21).read())
count_turned_on(mapping, 5)

136

In [312]:
% time count_turned_on(mapping, 18)

CPU times: user 5.36 s, sys: 71.3 ms, total: 5.43 s
Wall time: 5.43 s


1911767